# Scattershot Boson Sampling

Implementation of [this paper](https://arxiv.org/pdf/1305.4346.pdf) in Strawberry Fields

In [1]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from math import factorial, tanh

import strawberryfields as sf
from strawberryfields.ops import *

## Compute the circuit

### Constants

In [2]:
r_squeezing = 0.4
cutoff = 5

### Circuit

In [3]:
eng, q = sf.Engine(8)

In [4]:
with eng:
    S2gate(r_squeezing) | (q[0], q[4])
    S2gate(r_squeezing) | (q[1], q[5])
    S2gate(r_squeezing) | (q[2], q[6])
    S2gate(r_squeezing) | (q[3], q[7])

    Rgate(0.5719)  | q[0]
    Rgate(-1.9782) | q[1]
    Rgate(2.0603)  | q[2]
    Rgate(0.0644)  | q[3]

    BSgate(0.7804, 0.8578)  | (q[4], q[5])
    BSgate(0.06406, 0.5165) | (q[6], q[7])
    BSgate(0.473, 0.1176)   | (q[5], q[6])
    BSgate(0.563, 0.1517)   | (q[4], q[5])
    BSgate(0.1323, 0.9946)  | (q[6], q[7])
    BSgate(0.311, 0.3231)   | (q[5], q[6])
    BSgate(0.4348, 0.0798)  | (q[4], q[5])
    BSgate(0.4368, 0.6157)  | (q[6], q[7])

### Running

In [5]:
state = eng.run('fock', cutoff_dim=cutoff)

In [6]:
probs = state.all_fock_probs()

In [7]:
probs = probs.reshape(*[cutoff]*8)

In [8]:
np.sum(probs)

0.9894043325077697

## Analysis

### Get the unitary matrix

In [9]:
R = np.diag([np.exp(0.5719*1j),np.exp(-1.9782*1j),np.exp(2.0603*1j),np.exp(0.0644*1j)])

In [10]:
BSargs = [(0.7804, 0.8578),
          (0.06406, 0.5165),
          (0.473, 0.1176),
          (0.563, 0.1517),
          (0.1323, 0.9946),
          (0.311, 0.3231),
          (0.4348, 0.0798),
          (0.4368, 0.6157)
         ]

In [11]:
def get_BS_matrix(theta, phi):
    return [[np.cos(theta), - np.exp(-1j * phi) * np.sin(theta)], [np.exp(1j * phi) * np.sin(theta), np.cos(theta)]]

In [12]:
BS_matrices = np.array([get_BS_matrix(theta, phi) for (theta,phi) in BSargs])

In [13]:
UBS1 = sp.linalg.block_diag(*BS_matrices[0:2])
UBS2 = sp.linalg.block_diag([[1]], BS_matrices[2], [[1]])
UBS3 = sp.linalg.block_diag(*BS_matrices[3:5])
UBS4 = sp.linalg.block_diag([[1]], BS_matrices[5], [[1]])
UBS5 = sp.linalg.block_diag(*BS_matrices[6:8])

In [14]:
U = np.linalg.multi_dot([UBS5, UBS4, UBS3, UBS2, UBS1, R])

### Compute the theoretical probability

We use mostly the section V of [this paper](https://arxiv.org/pdf/1212.2240.pdf) as well as [the official tutorial](https://strawberryfields.readthedocs.io/en/latest/tutorials/tutorial_boson_sampling.html#boson-tutorial)

In [15]:
def perm(M):
    n_output = M.shape[0]
    n_input = M.shape[1]
    if n_output != n_input: # no conservation of photon number
        return 0
    n = n_input
    if n == 0:
        return 1
    d = np.ones(n)
    j =  0
    s = 1
    f = np.arange(n)
    v = M.sum(axis=0)
    p = np.prod(v)
    while (j < n-1):
        v -= 2*d[j]*M[j]
        d[j] = -d[j]
        s = -s
        prod = np.prod(v)
        p += s*prod
        f[0] = 0
        f[j] = f[j+1]
        f[j+1] = j+1
        j = f[0]    
    
    return p/2**(n-1)

In [16]:
def get_proba_output(U, input, output):
    list_rows = sum([[i] * output[i] for i in range(len(output))],[])
    list_columns = sum([[i] * input[i] for i in range(len(input))],[])
    U_st = U[:,list_columns][list_rows,:]
    perm_squared = np.abs(perm(U_st))**2
    denominator = np.prod([factorial(inp) for inp in input]) * np.prod([factorial(out) for out in output])
    return perm_squared / denominator

In [17]:
def get_proba_input(input):
    chi = np.tanh(r_squeezing)
    n = np.sum(input)
    m = len(input)
    return (1 - chi**2)**m * chi**(2*n)

In [18]:
def get_proba(U, result):
    input, output = result[0:4], result[4:8]
    return get_proba_output(U, input, output) * get_proba_input(input)

## Compare the simulation with the theory

In [19]:
print(get_proba(U, [0,0,0,0,0,0,0,0]))
print(probs[0,0,0,0,0,0,0,0])

0.535996373869716
0.5359963738697161


In [20]:
print(get_proba(U, [1,0,0,0,1,0,0,0]))
print(probs[1,0,0,0,1,0,0,0])

0.008821826459915328
0.008821826459915326
